# 6-3 주성분 분석

## - 주성분 분석 소개 (PCA)

> 참고: https://kolikim.tistory.com/27

#### 과일 사진 데이터 준비하기

준비된 과일 데이터는 사과, 바나나, 파인애플을 담고 있는 흑백사진이다. 
 - 데이터 출처 : https://github.com/rickiepark/hg-mldl/blob/master/fruits_300.npy

이 데이터는 넘파이 배열의 기본 저장 포맷인 npy 파일로 저장되어 있다.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
fruits = np.load('data/fruits_300.npy')

print(fruits.shape)

In [ ]:
fruits_2d = fruits.reshape(-1, 100*100)

### - PCA 클래스

 - n_components 매개변수에 주성분의 개수를 지정해야 한다. 
 - 비지도 학습이기 때문에 fit() 메서드에 타깃값을 제공하지 않는다.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
pca.fit(fruits_2d)

PCA 클래스가 찾은 주성분은 components_ 속성

In [ ]:
print(pca.components_.shape)


원본 데이터와 차원이 같으므로 주성분을 100x100 크기의 이미지처럼 출력해 볼 수 있다.


In [ ]:
def draw_fruits(arr, ratio=1):
    n = len(arr)    # n은 샘플 개수입니다
    # 한 줄에 10개씩 이미지를 그립니다. 샘플 개수를 10으로 나누어 전체 행 개수를 계산합니다. 
    rows = int(np.ceil(n/10))
    # 행이 1개 이면 열 개수는 샘플 개수입니다. 그렇지 않으면 10개입니다.
    cols = n if rows < 2 else 10
    fig, axs = plt.subplots(rows, cols, 
                            figsize=(cols*ratio, rows*ratio), squeeze=False)
    for i in range(rows):
        for j in range(cols):
            if i*10 + j < n:    # n 개까지만 그립니다.
                axs[i, j].imshow(arr[i*10 + j], cmap='gray_r')
            axs[i, j].axis('off')
    plt.show()

In [ ]:
draw_fruits(pca.components_.reshape(-1, 100, 100))

### 차원축소

주성분을 찾았으므로 원본 데이터를 주성분에 투영하여 특성의 개수를 10000개에서 50개로 줄일 수 있다. PCA의 trasnform() 메서드를 사용해 원본 데이터의 차원을 50으로 줄여보자.

In [ ]:
print(fruits_2d.shape)

In [ ]:
fruits_pca = pca.transform(fruits_2d)

In [ ]:
print(fruits_pca.shape)

## - 원본 데이터 재구성

PCA 클래스는 이를 위해 inverse_transform() 메서드를 제공한다.  50개의 차원으로 축소한 fruits_pca 데이터를 전달해 10000개의 특성을 복원하자.

In [ ]:
fruits_inverse = pca.inverse_transform(fruits_pca)
print(fruits_inverse.shape)

In [ ]:
fruits_reconstruct = fruits_inverse.reshape(-1, 100, 100)

In [ ]:
for start in [0, 100, 200]:
    draw_fruits(fruits_reconstruct[start:start+100])
    print("\n")

일부 흐리고 번진 부분이 있지만 모든 과일이 잘 복원 되었다. 

## - 설명된 분산

주성분이 원본 데이터의 분산을 얼마나 잘 나타내는지 기록한 값을 **설명된 분산** 이라고 한다. PCA 클래스의 explained_variance_ratio_에 각 주성분의 설명된 분산 비율이 기록되어 있다.

In [ ]:
print(np.sum(pca.explained_variance_ratio_))

In [ ]:
plt.plot(pca.explained_variance_ratio_)

## - 다른 알고리즘과 함께 사용하기

앞서 과일 사진 원본 데이터와 PCA로 축소한 데이터를 지도 학습에 적용

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

지도 학습 모델을 사용하려면 타깃값이 있어야 한다. 여기에서는 사과를 0, 파인애플을 1, 바나나를 2로 지정하자.

In [ ]:
target = np.array([0] * 100 + [1] * 100 + [2] * 100)

원본 데이터인 fruits_2d를 사용해 로지스틱 회귀 모델에서 성능을 가늠해 보기 위해 cross_validate()로 교차 검증을 수행하자.

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(lr, fruits_2d, target)
print(np.mean(scores['test_score']))
print(np.mean(scores['fit_time']))

교차 검증의 점수는 0.977 로 매우 높다. 특성이 10000개나 되기 때문에 300개의 샘플에서는 금방 과대적합된 모델을 만들기 쉽다. 

이 값을 PCA로 축소한 fruits_pca를 사용했을 때와 비교하자.

In [ ]:
scores = cross_validate(lr, fruits_pca, target)
print(np.mean(scores['test_score']))
print(np.mean(scores['fit_time']))

50개의 특성만 사용했는데 정확도가 100%이고 훈련 시간은 0.01초로 거의 98% 이상 감소했다. 

In [ ]:
(0.01 - 0.54)/0.54*100 - 100

설명된 분산의 50%에 달하는 주성분을 찾도록 PCA 모델을 만들어 보자.

In [ ]:
pca = PCA(n_components=0.5)
pca.fit(fruits_2d)

In [ ]:
print(pca.n_components_)

단 2개의 특성 만으로 분산의 50% 를 표현할 수 있다. 이 모델로 원본 데이터를 변환해보자. 주성분이 2개이므로 변환된 데이터의 크기는 (300,2)가 될 것이다.

In [ ]:
fruits_pca = pca.transform(fruits_2d)
print(fruits_pca.shape)

2개의 특성으로 교차 검증도 해보자.

In [ ]:
scores = cross_validate(lr, fruits_pca, target)
print(np.mean(scores['test_score']))
print(np.mean(scores['fit_time']))

### K-means

차원 축소된 데이터를 사용해 k-평균 알고리즘으로 클러스터를 찾아보자.

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=3, random_state=42)
km.fit(fruits_pca)

In [ ]:
print(np.unique(km.labels_, return_counts=True))

fruits_pca로 찾은 클러스터는 각각 110개, 99개, 91개의 샘플을 포함하고 있다. 이는 앞에서 원본 데이터를 사용했을 때와 거의 비슷한 결과이다. 이미지로 출력해보자.

In [ ]:
for label in range(0, 3):
    draw_fruits(fruits[km.labels_ == label])
    print("\n")

 - 파인애플은 사과와 조금 혼돈되는 면이 있다.

훈련 데이터의 차원을 줄이면 또 하나 얻을 수 있는 장점은 시각화이다. 3개 이하로 차원을 줄이면 화면에 출력하기 비교적 쉽다.

In [ ]:
for label in range(0, 3):
    data = fruits_pca[km.labels_ == label]
    plt.scatter(data[:,0], data[:,1])
plt.legend(['apple', 'banana', 'pineapple'])
plt.show()

각 클러스터의 산점도가 아주 잘 구분된다. 2개의 특성만을 사용했는데 교차 검증 점수가 99%가 나온 이유를 알 수 있다.

참고:

혼자서하는 ML/ 비지도 학습
